### esta porquería es un intento fallido de finetunnear un SentenceTransformer en Colab

In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.losses import SoftmaxLoss
from sentence_transformers.models import Dense
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd
import torch
import os


### Cargar tus datos de entrenamiento

In [2]:
# ========= Cargar y preparar el dataset =========
df = pd.read_csv("train.csv")

# Revisá los nombres de columnas
print(df.columns)

# Usá los nombres correctos según tu archivo
df = df.dropna(subset=['resume_text', 'job_description_text', 'label']) # Eliminar filas con NaN

# Convertir clases a números 'No Fit' = 0, 'Potential Fit' = 1, 'Good Fit' = 2
label_mapping = {'No Fit': 0, 'Potential Fit': 1, 'Good Fit': 2}
df['label'] = df['label'].map(label_mapping)
# Verificá que la conversión fue exitosa
print(df['label'].unique())


Index(['resume_text', 'job_description_text', 'label'], dtype='object')
[0 1 2]


In [3]:
print(df.head())

                                         resume_text  \
0  SummaryHighly motivated Sales Associate with e...   
1  Professional SummaryCurrently working with Cat...   
2  SummaryI started my construction career in Jun...   
3  SummaryCertified Electrical Foremanwith thirte...   
4  SummaryWith extensive experience in business/r...   

                                job_description_text  label  
0  Net2Source Inc. is an award-winning total work...      0  
1  At Salas OBrien we tell our clients that were ...      0  
2  Schweitzer Engineering Laboratories (SEL) Infr...      0  
3  Mizick Miller & Company, Inc. is looking for a...      0  
4  Life at Capgemini\nCapgemini supports all aspe...      0  


### Según ChatGPT:

| Modelo                 | Dim. | Velocidad | Precisión | Ideal para              |
| ---------------------- | ---- | --------- | --------- | ----------------------- |
| `all-MiniLM-L6-v2`     | 384  | Muy alta  | Buena     | Búsqueda semántica      |
| `all-mpnet-base-v2`    | 768  | Media     | Muy alta  | Clasificación o ranking |
| `paraphrase-MiniLM-v2` | 384  | Alta      | Media     | Detección de similitud  |


In [4]:
# ========= Configuración de GPU =========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Verificá si la GPU está disponible
if torch.cuda.is_available():
    print("GPU está disponible")
else:
    print("GPU no está disponible, usando CPU")

# Crear InputExamples
train_samples = [
    InputExample(texts=[row['resume_text'], row['job_description_text']], label=row['label'])
    for _, row in df.iterrows()
]

# A MODO DE PRUEBA, NOS VAMOS A QUEDAR CON 400 EJEMPLOS SOLAMENTE
train_samples = train_samples[:400]

GPU está disponible


### Cargar el modelo base

In [9]:
base_model = SentenceTransformer('all-mpnet-base-v2')


# Preparar DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)

# Cargar modelo base
num_classes = 3
embedding_dim = base_model.get_sentence_embedding_dimension()
classification_head = nn.Linear(embedding_dim, num_classes)

# SoftmaxLoss espera que uses SentenceTransformer completo
train_loss = losses.SoftmaxLoss(
    model=base_model,
    sentence_embedding_dimension=base_model.get_sentence_embedding_dimension(),
    num_labels=num_classes
)


### Finetuning

In [10]:
output_path = "output/fine_tuned_job_matcher"
os.makedirs(output_path, exist_ok=True)
os.environ["WANDB_DISABLED"] = "true"

base_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=4,
    warmup_steps=10,
    show_progress_bar=True,
    output_path=output_path,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 108.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 48.12 MiB is free. Process 4745 has 14.69 GiB memory in use. Of the allocated memory 14.46 GiB is allocated by PyTorch, and 110.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)